In [1]:
# set working directory to same place ass app.py to import programs the same way as the app
import os
current_directory = os.getcwd()
if 'notebooks' in current_directory:
    parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
    os.chdir(parent_directory)
os.getcwd()

'/Users/elisealstad/Desktop/Code/mybook-dashboard'

In [2]:
import pandas as pd 
import json
import numpy as np
pd.set_option('display.max_columns',100)


# Importing the data
- importing topics in json file
- import book df 

In [3]:
# mybooks = pd.read_csv('assets/goodreads_library_export.csv')

In [4]:
# mybooks.head(25).to_csv('assets/goodreads_library_export(1).csv')

In [5]:
# with open('assets/my_topics.json') as file:
#     json_data = json.load(file)

# # # Convert JSON data to a dictionary
# my_read_topics = dict(my_read_topics)

In [6]:
from apps.dataimport import *

mybooks = pd.read_csv('assets/goodreads_library_export.csv')
myreads = mybooks.loc[mybooks['Exclusive Shelf'] == "read"]
my_read_topics, myreads = dataprep(myreads.head(15))

# # GET DATA - topics json
# import json
# # Load JSON data from a file

nmy_read_topics, nmyreads = dataprep(myreads)

uploadtxt_suc = "Success, your data have been uploaded and the figures updated!"
   


# Import good reads book from pickle

In [7]:
mybooks = pd.read_pickle('assets/my_books.pkl')

# # filer only books I have read
myreads = mybooks.query('Exclusive_Shelf == "read"').copy()
to_read = mybooks.query("Exclusive_Shelf == 'to-read'").copy()

# Fill missing date read with date added
myreads['Date_Read'] = myreads['Date_Read'].fillna(myreads['Date_Added'])
myreads = myreads.sort_values(by='Date_Read')

### Import topics dict from json file

In [8]:
import json

# Load JSON data from a file
with open('assets/my_topics.json') as file:
    json_data = json.load(file)

# Convert JSON data to a dictionary
my_topics = dict(json_data)

In [9]:
# creating dictionaries for read and want-to-read books
my_read_topics = {k: v for k, v in my_topics.items() if k in myreads.Title.to_list()}
my_want_titles = list(set(list(my_topics.keys())) - set(myreads.Title.to_list()))
my_want_topics = {k: v for k, v in my_topics.items() if k in my_want_titles}

In [10]:
import plotly.express as px
from collections import Counter

word_counts_dict = Counter()

# Count the occurrences of each word in the values of the original dictionary
for values in my_read_topics.values():
    for value in values:
        for word in value.replace(', ', ',').split(','):
            word_counts_dict[word.lower()] += 1

df_word_counts = pd.DataFrame(list(word_counts_dict.items()), columns=['Word', 'Count'])
df_word_counts = df_word_counts.sort_values(by='Count', ascending=False).reset_index().head(45)

fig = px.treemap(
    df_word_counts,
    path=['Word'],
    values='Count',
    custom_data=['Count'] )

# Update the layout of the tree map
fig.update_layout(
    title='Most common topics'
)
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>Count: %{customdata[0]}'
)
# Show the plot
fig.show()

# Topics visualization 

In [11]:
from apps.viz import tree_topics
fig = tree_topics(my_read_topics)
fig.show()

# vizualising publication year

In [12]:
from apps.viz import viz_pub_year
fig = viz_pub_year(myreads)
fig.show()

# Vizualising timeline of books read. 

In [13]:
from apps.viz import viz_year_read
viz_year_read(myreads).show()

/Users/elisealstad/Desktop/Code/mybook-dashboard/apps/viz.py:116: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



# Number of pages and read count

Visualising the most read page count of books

In [14]:
from apps.viz import visualize_page_categories

# Assuming you have a DataFrame 'myreads' with 'Page_Cat' column

visualize_page_categories(myreads, 'My_Rating', 'How do I rate my books?<br><span style="font-size: 8px;">Number of books per Ratings category</span>', 'Goodreads rating')



ImportError: cannot import name 'visualize_page_categories' from 'apps.viz' (/Users/elisealstad/Desktop/Code/mybook-dashboard/apps/viz.py)

In [ ]:
# Assuming you have a DataFrame 'myreads' with 'Page_Cat' column

visualize_page_categories(myreads, 'Page_Cat', 'How long are the books I read?<br><span style="font-size: 8px;">Number of books per Page Count Category</span>', 'Page Count Category')

# Vizualise top categories and languages
I am using a pie chart here because I excpect few values in each variable and a large discrepancy.

In [ ]:
from apps.viz import viz_top_values

In [ ]:
viz_top_values(mybooks['Language'], top_n=5)

In [ ]:
viz_top_values(mybooks['Categories'], top_n=10)

## Rating visualised 

In [ ]:
from apps.viz import book_ratings
# Call the function for top-rated books
fig_top_rated = book_ratings(myreads, 'Top Rated Books', top_rated=True)

# Call the function for bottom-rated books
fig_bottom_rated = book_ratings(myreads, 'Bottom Rated Books', top_rated=False)

# Show both figures
fig_top_rated.show()
fig_bottom_rated.show()



In [ ]:


from apps.viz import create_rating_table

# Call the function to create the rating table
rating_table = create_rating_table(myreads)

# Display the table
rating_table.show()


In [ ]:
from apps.viz import create_author_table
# Call the function to create the author table
author_table = create_author_table(myreads)

# Display the table
author_table.show()

# This year in books

In [ ]:
from datetime import datetime
datetime.today().year

2023

In [ ]:
# Filter on books read this year using datetime to find todays year
from datetime import datetime
today_year = datetime.today().year
print(f"This year you have read {len(myreads.query('Year == @today_year'))} books. Totaling {f'{(myreads.Number_of_Pages.sum().astype(int)):,}'} pages read.")

This year you have read 51 books. Totaling 62,094 pages read.


# Word cloud from Goodreads book Description

In [ ]:
import numpy as np
import plotly.express as px
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords

def desc_tree(Description):
    from collections import Counter
    stop_words = nltk.download('stopwords')
    stopwords_dict = Counter(stopwords.words('english'))
    stopwords_dict.update({'author', 'S', 'will','new','york','time','book','novel', 'read', 'day', 'make','year', 'one', 'times', 'times, of', 's', 'award','author','new','york','selling','story','t','1','og'})
        
    descriptions_all = ' '.join(Description.dropna()).lower()

    descriptions_all = re.findall(r'\b\w+\b', descriptions_all.lower())

    # Remove stop words from the list of words
    filtered_words = [word for word in descriptions_all if word not in stopwords_dict]

    word_counts = Counter(filtered_words)

    # Create a DataFrame from the word counts
    word_counts_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Count'])
    # Create a tree map using Plotly Express
    fig = px.treemap(word_counts_df.sort_values(by='Count', ascending=False).head(55), path=['Word'], values='Count', custom_data=['Count'] )

    # Update the layout of the tree map
    fig.update_layout(
        title='Most common words found in book descriptions'
    )
    fig.update_traces(
        hovertemplate='<b>%{label}</b><br>Count: %{customdata[0]}'
    )

    return fig

desc_tree(mybooks['Description'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/elisealstad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
